In [1]:
import mysql.connector


In [2]:
conn = mysql.connector.connect(host = "sql8.freesqldatabase.com", username="sql8713045", password="VZyDbMNYZZ",
                       database = "sql8713045", port=3306)

In [3]:
cursor = conn.cursor()


In [4]:
cursor.execute("""
ALTER TABLE Loans 
ADD FOREIGN KEY (BookID)
REFERENCES Books(BookID); """)


In [9]:
cursor.execute("""
ALTER TABLE Loans 
ADD FOREIGN KEY (PatronID)
REFERENCES Patrons(PatronID); """)

In [12]:
 #find the number of availalbe copies of Dracula

cursor.execute(""" 
SELECT Title, count(Title) as available_copies FROM Books as b 
LEFT JOIN Loans as l ON b.BookID = l.BookID
WHERE b.Title = "Dracula" AND l.ReturnedDate IS NOT NULL
GROUP BY Title; """)

In [13]:
available_dracula_copies = cursor.fetchall()
available_dracula_copies

[('Dracula', 31)]

In [19]:
# check total copies of the book 

cursor.execute(""" 
SELECT Title, count(Title) as total_copies FROM Books as b 
INNER JOIN Loans as l ON b.BookID = l.BookID
WHERE b.Title = "Dracula"
GROUP BY Title; """)

In [20]:
total_dracula_copies = cursor.fetchall()
total_dracula_copies

[('Dracula', 32)]

In [22]:
# current total loans of the book 

cursor.execute(""" 
SELECT Title, count(Title) as total_loans FROM Books as b 
INNER JOIN Loans as l ON b.BookID = l.BookID
WHERE b.Title = "Dracula" AND l.ReturnedDate IS NULL
GROUP BY Title; """)


In [23]:
total_loans = cursor.fetchall()
total_loans

[('Dracula', 1)]

In [25]:
# total available book 

cursor.execute(""" 
SELECT Title, count(Title) as total_available_books FROM Books as b 
LEFT JOIN Loans as l ON b.BookID = l.BookID
WHERE l.ReturnedDate IS NOT NULL
GROUP BY Title; """)

In [26]:
total_available_books = cursor.fetchall()
total_available_books

[("A Doll's House", 28),
 ('A Modest Proposal', 20),
 ('A Study in Scarlet', 30),
 ('A Tale of Two Cities', 37),
 ('Adventures of Huckleberry Finn', 23),
 ("Alice's Adventures in Wonderland", 38),
 ('Also sprach Zarathustra', 28),
 ('An Occurrence at Owl Creek Bridge', 32),
 ('Anna Karenina', 24),
 ('Anne of Green Gables', 24),
 ('Candide', 21),
 ('David Copperfield', 24),
 ('Don Quixote', 22),
 ('Dracula', 31),
 ('Dubliners', 26),
 ('Emma', 35),
 ('Frankenstein; Or, The Modern Prometheus', 53),
 ('Great Expectations', 29),
 ("Grimms' Fairy Tales", 32),
 ("Gulliver's Travels into Several Remote Nations of the World", 31),
 ('Hard Times', 25),
 ('Heart of Darkness', 19),
 ('Jane Eyre: An Autobiography', 35),
 ('Les Misérables', 25),
 ('Little Women', 30),
 ('Metamorphosis', 29),
 ('Moby Dick; Or, The Whale', 19),
 ('Oliver Twist', 21),
 ('On the origin of species', 31),
 ("Pascal's Pensées", 17),
 ('Peter Pan', 29),
 ('Pride and Prejudice', 28),
 ('Pygmalion', 21),
 ('Second Treatise of

In [29]:
# Add new books to the library                        

cursor.execute(""" 
INSERT INTO Books VALUES 
(201, "A Game Of Thrones", "George R. R. Martin", 1996, 2719805889), 
(202, "Terriens (PHOTOGRAPHIE)", "Richard Kalvar", 2007, 2081201135); """)

IntegrityError: 1062 (23000): Duplicate entry '201' for key 'PRIMARY'

In [30]:
# Check out Books 

cursor.execute(""" 
INSERT INTO Patrons Values ("Shahd", "Hatem", "shahdhdaoud@gmail.com", 201); """)

cursor.execute(""" 
INSERT INTO Loans (LoanID, BookID, PatronID, LoanDate, DueDate) 
VALUES 
(2001, 201, (SELECT PatronID FROM Patrons WHERE Email = "shahdhdaoud@gmail.com") , "2024-06-09", "2024-07-09"), 
(2002, 202, (SELECT PatronID FROM Patrons WHERE Email = "ccalver8@samoca.org"), "2024-06-10", "2024-07-10"); """)

[]

In [37]:
# Check books for Due back                             
# generate a report of books due back on July 13, 2020 
# with patron contact information   

cursor.execute(""" 
SELECT b.Title, CONCAT(p.FirstName, " ", p.LastName) as full_name, p.Email, l.DueDate FROM Books as b
INNER JOIN Loans as l ON b.BookID = l.BookID
INNER JOIN Patrons as p ON p.PatronID = l.PatronID
WHERE DueDate = "2020-07-13"; """)

In [38]:
due_books = cursor.fetchall()
due_books

[('Second Treatise of Government',
  'Anthe Dinjes',
  'adinjesh@samoca.org',
  '2020-07-13'),
 ('The Prophet', 'Lydie Awmack', 'lawmack1a@samoca.org', '2020-07-13'),
 ('Dracula', 'Cornelia Koppe', 'ckoppe1m@landonhotel.com', '2020-07-13'),
 ('The Count of Monte Cristo',
  'Kala Simonian',
  'ksimonian2q@landonhotel.com',
  '2020-07-13')]

In [ ]:
# Return books to the library                         

cursor.execute(""" UPDATE Loans 
SET ReturnedDate = CASE LoanID
WHEN 2001 THEN "2024-07-09"
WHEN 2002 THEN "2024-07-10"
ELSE ReturnedDate
END
WHERE LoanID IN(2001, 2002); """)

In [39]:
# Encourage Patrons to check out books                
# generate a report of showing 10 patrons who have
# checked out the fewest books.

cursor.execute(""" 
SELECT p.PatronID, CONCAT(p.FirstName, " ", p.LastName) as full_name, count(l.LoanID) as borrowed_books FROM Loans as l
INNER JOIN Patrons as p ON l.PatronID = p.PatronID
GROUP BY p.PatronID
ORDER BY borrowed_books
LIMIT 10; """)

In [40]:
fewest_books = cursor.fetchall()
fewest_books

[(69, 'Alaster Ruggles', 9),
 (83, 'Caril Matejic', 9),
 (73, 'Merle Sukbhans', 11),
 (64, 'Norby Sleight', 12),
 (94, 'Curr Fittall', 12),
 (50, 'Jack Vaan', 13),
 (3, 'Aleen Fasey', 14),
 (7, 'Cammi Kynett', 14),
 (12, 'Orton Stavers', 14),
 (16, 'Carla Swinfen', 14)]

In [47]:
# Find books to feature for an event                  
# create a list of books from 1890s that are currently available                                    

cursor.execute(""" 
SELECT b.Title, b.Published, count(b.Title) as total_available_books FROM Books as b 
INNER JOIN Loans as l ON b.BookID = l.BookID
WHERE l.ReturnedDate IS NOT NULL AND b.Published BETWEEN 1890 AND 1899
GROUP BY Title
ORDER BY Published; """)

In [48]:
nineties_available_books = cursor.fetchall()
nineties_available_books

[('The Sign of the Four', 1890, 25),
 ('The Picture of Dorian Gray', 1890, 39),
 ('An Occurrence at Owl Creek Bridge', 1890, 32),
 ('The Adventures of Sherlock Holmes', 1892, 26),
 ('The Yellow Wallpaper', 1892, 31),
 ('The Jungle Book', 1894, 28),
 ('The Importance of Being Earnest: A Trivial Comedy for Serious People',
  1895,
  31),
 ('The Time Machine', 1895, 27),
 ('Les Misérables', 1895, 25),
 ('Also sprach Zarathustra', 1896, 28),
 ('Dracula', 1897, 31),
 ('The War of the Worlds', 1898, 43),
 ('The Turn of the Screw', 1898, 36),
 ('The Awakening, and Selected Short Stories', 1899, 43),
 ('Heart of Darkness', 1899, 19)]

In [49]:
# Book Statistics 
# create a report to show how many books were published each year.    


cursor.execute(""" 
SELECT Published, COUNT(DISTINCT(Title)) AS TotalNumberOfPublishedBooks
FROM Books
GROUP BY Published
ORDER BY TotalNumberOfPublishedBooks DESC; """)

In [50]:
yearly_published_books = cursor.fetchall()
yearly_published_books

[(1890, 3),
 (1895, 3),
 (1869, 2),
 (1898, 2),
 (1899, 2),
 (1847, 2),
 (1884, 2),
 (1850, 2),
 (1911, 2),
 (1892, 2),
 (1859, 2),
 (1922, 2),
 (1791, 1),
 (1996, 1),
 (1811, 1),
 (1876, 1),
 (2007, 1),
 (1813, 1),
 (1878, 1),
 (1900, 1),
 (1815, 1),
 (1879, 1),
 (1901, 1),
 (1818, 1),
 (1880, 1),
 (1902, 1),
 (1839, 1),
 (1883, 1),
 (1905, 1),
 (1906, 1),
 (1849, 1),
 (1886, 1),
 (1908, 1),
 (1375, 1),
 (1887, 1),
 (1532, 1),
 (1851, 1),
 (1889, 1),
 (1913, 1),
 (1605, 1),
 (1852, 1),
 (1914, 1),
 (1670, 1),
 (1854, 1),
 (1915, 1),
 (1689, 1),
 (1858, 1),
 (1894, 1),
 (1920, 1),
 (1726, 1),
 (1921, 1),
 (1729, 1),
 (1861, 1),
 (1896, 1),
 (1759, 1),
 (1865, 1),
 (1897, 1),
 (1923, 1)]

In [51]:
# Book Statistics                                           
# create a report to show 5 most popular Books to check out 

cursor.execute(""" 
SELECT b.Title, b.Author, b.Published, COUNT(b.Title) AS TotalTimesOfLoans
FROM Books b
JOIN Loans l
ON b.BookID = l.BookID
GROUP BY b.Title
ORDER BY 4 DESC
LIMIT 5; """)

In [52]:
five_popular_books = cursor.fetchall()
five_popular_books

[('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  1818,
  53),
 ('The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 43),
 ('The War of the Worlds', 'H. G. Wells', 1898, 43),
 ('The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 42),
 ('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 39)]